In [33]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [5]:
print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("../data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("../data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("../data/store.csv")

train.fillna(1, inplace=True)
test.fillna(1, inplace=True)
train = train[train["Open"] != 0]
train = train[train["Sales"] > 0]

train = pd.merge(train, store, on='Store')

Load the training, test and store data using pandas


In [ ]:
#     store['Sales25th'] = 0
#     store['Sales50th'] = 0
#     store['Sales75th'] = 0
#     store['SalesMedian'] = 0

#     for store_id in train.Store.unique():
#         Sales25th = train[train.Store == store_id].Sales.quantile(0.25)
#         Sales50th = train[train.Store == store_id].Sales.quantile(0.50)
#         Sales75th = train[train.Store == store_id].Sales.quantile(0.75)
#         SalesMedian = train[train.Store == store_id].Sales.median()

#         store.loc[store.Store == store_id, 'Sales25th'] = Sales25th
#         store.loc[store.Store == store_id, 'Sales50th'] = Sales50th
#         store.loc[store.Store == store_id, 'Sales75th'] = Sales75th
#         store.loc[store.Store == store_id, 'SalesMedian'] = SalesMedian

In [19]:
def build_features_store(store, train):
    train = train.copy()
    train['Year'] = train.Date.dt.year
    train['Month'] = train.Date.dt.month
    train['Day'] = train.Date.dt.day
    train['DayOfWeek'] = train.Date.dt.dayofweek
    train['WeekOfYear'] = train.Date.dt.weekofyear
    
    store_ids = store.Store.unique()
    for store_id in store_ids:
        for day in train.DayOfWeek.unique():
            keyword = 'SalesDayOfWeek'
            name_mean = keyword + str(day) + "_Mean"
            name_median = keyword + str(day) + "_Median"
            name_25 = keyword + str(day) + "_25th"
            name_75 = keyword + str(day) + "_75th"
            store[name_mean] = 0
            store[name_median] = 0
            store[name_25] = 0
            store[name_75] = 0
        for day in train.Day.unique():
            keyword = 'SalesDay'
            name_mean = keyword + str(day) + "_Mean"
            name_median = keyword + str(day) + "_Median"
            name_25 = keyword + str(day) + "_25th"
            name_75 = keyword + str(day) + "_75th"
            store[name_mean] = 0
            store[name_median] = 0
            store[name_25] = 0
            store[name_75] = 0
        for week in train.WeekOfYear.unique():
            keyword = 'SalesWeekOfYear'
            name_mean = keyword + str(week) + "_Mean"
            name_median = keyword + str(week) + "_Median"
            name_25 = keyword + str(week) + "_25th"
            name_75 = keyword + str(week) + "_75th"
            store[name_mean] = 0
            store[name_median] = 0
            store[name_25] = 0
            store[name_75] = 0
        for year in train.Year.unique():
            keyword = 'SalesYear'
            name_mean = keyword + str(year) + "_Mean"
            name_median = keyword + str(year) + "_Median"
            name_25 = keyword + str(year) + "_25th"
            name_75 = keyword + str(year) + "_75th"
            store[name_mean] = 0
            store[name_median] = 0
            store[name_25] = 0
            store[name_75] = 0
            
    
    for store_id in store_ids:
        for day in train.DayOfWeek.unique():
            keyword = 'SalesDayOfWeek'
            name_mean = keyword + str(day) + "_Mean"
            name_median = keyword + str(day) + "_Median"
            name_25 = keyword + str(day) + "_25th"
            name_75 = keyword + str(day) + "_75th"
            store.loc[store.Store == store_id, name_mean] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.mean()
            store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.median()
            store.loc[store.Store == store_id, name_25] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.quantile(0.25)
            store.loc[store.Store == store_id, name_75] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.quantile(0.75)
        for day in train.Day.unique():
            keyword = 'SalesDay'
            name_mean = keyword + str(day) + "_Mean"
            name_median = keyword + str(day) + "_Median"
            name_25 = keyword + str(day) + "_25th"
            name_75 = keyword + str(day) + "_75th"
            store.loc[store.Store == store_id, name_mean] = train[(train.Store == store_id) & (train.Day == day)].Sales.mean()
            store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.Day == day)].Sales.median()
            store.loc[store.Store == store_id, name_25] = train[(train.Store == store_id) & (train.Day == day)].Sales.quantile(0.25)
            store.loc[store.Store == store_id, name_75] = train[(train.Store == store_id) & (train.Day == day)].Sales.quantile(0.75)
        for week in train.WeekOfYear.unique():
            keyword = 'SalesWeekOfYear'
            name_mean = keyword + str(week) + "_Mean"
            name_median = keyword + str(week) + "_Median"
            name_25 = keyword + str(week) + "_25th"
            name_75 = keyword + str(week) + "_75th"
            store.loc[store.Store == store_id, name_mean] = train[(train.Store == store_id) & (train.WeekOfYear == week)].Sales.mean()
            store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.WeekOfYear == week)].Sales.median()
            store.loc[store.Store == store_id, name_25] = train[(train.Store == store_id) & (train.WeekOfYear == week)].Sales.quantile(0.25)
            store.loc[store.Store == store_id, name_75] = train[(train.Store == store_id) & (train.WeekOfYear == week)].Sales.quantile(0.75)
        for year in train.Year.unique():
            keyword = 'Salesyear'
            name_mean = keyword + str(year) + "_Mean"
            name_median = keyword + str(year) + "_Median"
            name_25 = keyword + str(year) + "_25th"
            name_75 = keyword + str(year) + "_75th"
            store.loc[store.Store == store_id, name_mean] = train[(train.Store == store_id) & (train.Year == year)].Sales.mean()
            store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.Year == year)].Sales.median()
            store.loc[store.Store == store_id, name_25] = train[(train.Store == store_id) & (train.Year == year)].Sales.quantile(0.25)
            store.loc[store.Store == store_id, name_75] = train[(train.Store == store_id) & (train.Year == year)].Sales.quantile(0.75)
        
    store.fillna(0, inplace=True)        
    
build_features_store(store, train)
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Salesyear2015_25th,Salesyear2015_75th,Salesyear2014_Mean,Salesyear2014_Median,Salesyear2014_25th,Salesyear2014_75th,Salesyear2013_Mean,Salesyear2013_Median,Salesyear2013_25th,Salesyear2013_75th
0,1,c,a,1270,9,2008,0,0,0,0,...,3886.00,5032.50,4730.719472,4607.0,3908.50,5286.00,4921.254125,4804.0,4146.00,5572.00
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",...,3849.50,6202.50,4988.263158,4822.0,3891.25,6090.50,4895.276316,4725.0,3807.25,6068.00
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",...,5407.00,8301.50,6864.069536,6587.0,5123.75,8430.50,7047.235099,6648.0,5291.25,8685.75
3,4,c,c,620,9,2009,0,0,0,0,...,8546.75,10696.00,9776.279605,9509.0,8421.50,10730.25,9383.773026,9188.5,7973.50,10248.00
4,5,a,a,29910,4,2015,0,0,0,0,...,3712.50,5720.25,4657.168874,4553.5,3623.00,5782.25,4718.365449,4686.0,3636.00,5725.00


In [34]:
store.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,SalesDayOfWeek4Mean,SalesDayOfWeek3Mean,SalesDayOfWeek2Mean,SalesDayOfWeek1Mean,SalesDayOfWeek0Mean,SalesDayOfWeek5Mean,SalesDayOfWeek6Mean,SalesDayOfWeek4_Mean,SalesDayOfWeek4_Median,SalesDayOfWeek4_25th,SalesDayOfWeek4_75th,SalesDayOfWeek3_Mean,SalesDayOfWeek3_Median,SalesDayOfWeek3_25th,SalesDayOfWeek3_75th,SalesDayOfWeek2_Mean,SalesDayOfWeek2_Median,SalesDayOfWeek2_25th,SalesDayOfWeek2_75th,SalesDayOfWeek1_Mean,SalesDayOfWeek1_Median,SalesDayOfWeek1_25th,SalesDayOfWeek1_75th,SalesDayOfWeek0_Mean,SalesDayOfWeek0_Median,SalesDayOfWeek0_25th,SalesDayOfWeek0_75th,SalesDayOfWeek5_Mean,SalesDayOfWeek5_Median,SalesDayOfWeek5_25th,SalesDayOfWeek5_75th,SalesDayOfWeek6_Mean,SalesDayOfWeek6_Median,SalesDayOfWeek6_25th,SalesDayOfWeek6_75th,SalesWeekOfYear6_Mean,SalesWeekOfYear6_Median,SalesWeekOfYear6_25th,SalesWeekOfYear6_75th,SalesDay31_Mean,SalesDay31_Median,SalesDay31_25th,SalesDay31_75th,SalesDay30_Mean,SalesDay30_Median,SalesDay30_25th,SalesDay30_75th,SalesDay29_Mean,SalesDay29_Median,SalesDay29_25th,SalesDay29_75th,SalesDay28_Mean,SalesDay28_Median,SalesDay28_25th,SalesDay28_75th,SalesDay27_Mean,SalesDay27_Median,SalesDay27_25th,SalesDay27_75th,SalesDay25_Mean,SalesDay25_Median,SalesDay25_25th,SalesDay25_75th,SalesDay24_Mean,SalesDay24_Median,SalesDay24_25th,SalesDay24_75th,SalesDay23_Mean,SalesDay23_Median,SalesDay23_25th,SalesDay23_75th,SalesDay22_Mean,SalesDay22_Median,SalesDay22_25th,SalesDay22_75th,SalesDay21_Mean,SalesDay21_Median,SalesDay21_25th,SalesDay21_75th,SalesDay20_Mean,SalesDay20_Median,SalesDay20_25th,SalesDay20_75th,SalesDay18_Mean,SalesDay18_Median,SalesDay18_25th,SalesDay18_75th,SalesDay17_Mean,SalesDay17_Median,SalesDay17_25th,SalesDay17_75th,SalesDay16_Mean,SalesDay16_Median,SalesDay16_25th,SalesDay16_75th,SalesDay15_Mean,SalesDay15_Median,SalesDay15_25th,SalesDay15_75th,SalesDay14_Mean,SalesDay14_Median,SalesDay14_25th,SalesDay14_75th,SalesDay13_Mean,SalesDay13_Median,SalesDay13_25th,SalesDay13_75th,SalesDay11_Mean,SalesDay11_Median,SalesDay11_25th,SalesDay11_75th,SalesDay10_Mean,SalesDay10_Median,SalesDay10_25th,SalesDay10_75th,SalesDay9_Mean,SalesDay9_Median,SalesDay9_25th,SalesDay9_75th,SalesDay8_Mean,SalesDay8_Median,SalesDay8_25th,SalesDay8_75th,SalesDay7_Mean,SalesDay7_Median,SalesDay7_25th,SalesDay7_75th,SalesDay6_Mean,SalesDay6_Median,SalesDay6_25th,SalesDay6_75th,SalesDay4_Mean,SalesDay4_Median,SalesDay4_25th,SalesDay4_75th,SalesDay3_Mean,SalesDay3_Median,SalesDay3_25th,SalesDay3_75th,SalesDay2_Mean,SalesDay2_Median,SalesDay2_25th,SalesDay2_75th,SalesDay1_Mean,SalesDay1_Median,SalesDay1_25th,SalesDay1_75th,SalesDay26_Mean,SalesDay26_Median,SalesDay26_25th,SalesDay26_75th,SalesDay19_Mean,SalesDay19_Median,SalesDay19_25th,SalesDay19_75th,SalesDay12_Mean,SalesDay12_Median,SalesDay12_25th,SalesDay12_75th,SalesDay5_Mean,SalesDay5_Median,SalesDay5_25th,SalesDay5_75th,SalesWeekOfYear31_Mean,SalesWeekOfYear31_Median,SalesWeekOfYear31_25th,SalesWeekOfYear31_75th,SalesWeekOfYear30_Mean,SalesWeekOfYear30_Median,SalesWeekOfYear30_25th,SalesWeekOfYear30_75th,SalesWeekOfYear29_Mean,SalesWeekOfYear29_Median,SalesWeekOfYear29_25th,SalesWeekOfYear29_75th,SalesWeekOfYear28_Mean,SalesWeekOfYear28_Median,SalesWeekOfYear28_25th,SalesWeekOfYear28_75th,SalesWeekOfYear27_Mean,SalesWeekOfYear27_Median,SalesWeekOfYear27_25th,SalesWeekOfYear27_75th,SalesWeekOfYear26_Mean,SalesWeekOfYear26_Median,SalesWeekOfYear26_25th,SalesWeekOfYear26_75th,SalesWeekOfYear25_Mean,SalesWeekOfYear25_Median,SalesWeekOfYear25_25th,SalesWeekOfYear25_75th,SalesWeekOfYear24_Mean,SalesWeekOfYear24_Median,SalesWeekOfYear24_25th,SalesWeekOfYear24_75th,SalesWeekOfYear23_Mean,SalesWeekOfYear23_Median,SalesWeekOfYear23_25th,SalesWeekOfYear23_75th,SalesWeekOfYear22_Mean,SalesWeekOfYear22_Median,SalesWeekOfYear22_25th,SalesWeekOfYear22_75th,SalesWeekOfYear21_Mean,SalesWeekOfYear21_Median,SalesWeekOfYear21_25th,SalesWeekOfYear21_75th,SalesWe

In [47]:
store.to_csv("../data/store_features.pd", index=False)